In [ ]:
# Set study variables
study_home = '/data/perlman/moochie/user_data/CamachoCat/5YOP'
input_dir = study_home + '/hcp_proc'
output_dir = study_home + '/proc/rest_mapping'
subject_id = '3000'

In [ ]:
# load data
import hcp_utils as hcp
import nibabel as nib
import numpy as np

func_data_file = input_dir +'/{0}/MNINonLinear/Results/movie/movie_Atlas.dconn.nii'.format(subject_id)
func_img = nib.load(func_data_file)
hemi2_func_data = func_img.get_fdata()
hemi1_func_data = hemi2_func_data[:29696,:29696]
hemi2_func_data = hemi2_func_data[29696:59412,29696:59412]

In [ ]:
labels = [str(a) for a in range(0,hemi1_func_data.shape[0])]

nodes_weights = open('movie_h1_nodes_nothresh_weights.txt','a')
y=0
while y < hemi1_func_data.shape[0]:
    for x in range(y,hemi1_func_data.shape[0]-1):
        #z = 0.5*(np.log10(1+hemi1_func_data[x,x+1]) - np.log10(1-hemi1_func_data[x,x+1]))
        #if z>0.05:
        nodes_weights.write('{0} {1} {2}\n'.format(labels[x],labels[x+1],hemi1_func_data[x,x+1]))
    y = y+1
    
    
nodes_weights = open('movie_h2_nodes_nothresh_weights.txt','a')
labels = [str(a + 29696) for a in range(0,hemi2_func_data.shape[0])]   
y=0
while y < hemi2_func_data.shape[0]:
    for x in range(y,hemi2_func_data.shape[0]-1):
        #z = 0.5*(np.log10(1+hemi2_func_data[x,x+1]) - np.log10(1-hemi2_func_data[x,x+1]))
        #if z>0.05:
        nodes_weights.write('{0} {1} {2}\n'.format(labels[x],labels[x+1],hemi2_func_data[x,x+1]))
    y = y+1

In [ ]:
import nibabel as nib
import numpy as np
orig_img = nib.load('/data/perlman/moochie/user_data/CamachoCat/5YOP/hcp_proc/3000/MNINonLinear/Results/firstvol/firstvol_Atlas.dtseries.nii')
orig_data = orig_img.get_fdata()
print(orig_data.shape)
new_data = np.zeros(orig_data.shape)

In [ ]:
from igraph import Graph
import random
random.seed(540000)
gnodes = Graph.Read_Ncol('fixation_h1_nodes_nothresh_weights.txt', names=('node1','node2','weight'), weights=True)
mods1 = gnodes.community_infomap(edge_weights='weight',trials=8) 

In [ ]:
import numpy as np
net1_labels = np.array(mods1.membership) + 1
n,c=np.unique(net1_labels,return_counts=True)
print(n)
print(c)

In [ ]:
for i in range(0,len(n)):
    if c[i]<100:
        net1_labels[net1_labels==n[i]]=0

In [ ]:
print(net1_labels.shape)
new_data[0,0:29696] = net1_labels

In [ ]:
new_img = nib.cifti2.cifti2.Cifti2Image(new_data, header = orig_img.header, nifti_header=orig_img.nifti_header)
nib.save(new_img, 'fixation_left_hemi.dtseries.nii')

In [ ]:
gnodes2 = Graph.Read_Ncol('fixation_h2_nodes_nothresh_weights.txt', names=('node1','node2','weight'), weights=True, directed=True)
mods2 = gnodes2.community_infomap(edge_weights='weight',trials=8) 

In [ ]:
net2_labels = np.array(mods2.membership) + 1
n,c=np.unique(net2_labels,return_counts=True)
print(n)
print(c)

In [ ]:
for i in range(0,len(n)):
    if c[i]<100:
        net2_labels[net2_labels==n[i]]=0

In [ ]:
print(net2_labels.shape)
new_data[0,29696:59412] = net2_labels

In [ ]:
new_img = nib.cifti2.cifti2.Cifti2Image(new_data, header = orig_img.header, nifti_header=orig_img.nifti_header)
nib.save(new_img, 'fixation_right_hemi.dtseries.nii')